In [ ]:
import numpy as np
import os
import h5py
import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.particle_fitting as fit
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display

# Set environment variables for processing
os.environ['NUMEXPR_MAX_THREADS'] = '32'
%matplotlib notebook

# File dialog to select the input file
file = img.fileDialog(os.getcwd())
assert os.path.isfile(file), f'File "{file}" does not exist'

# General parameters
frame_range = [0, 10000]  # Restrict analysis to certain frames
navg = 1  # Frame averaging before background removal
assert len(frame_range) == 2 or len(frame_range) == 0, 'frame_range must be [] or [int, int]'

# Background removal
mode = 'continuous_median'  # Background removal strategy
window_length = 1001  # Median window length

# Choose between CPU and GPU processing
parallel = True  # Use CPU
GPU = False  # Use GPU (requires CUDA and PyTorch)

# Apply continuous median background removal
frames, file = img.mp_reader(
    batch_mode=True,
    file_to_load=file,
    frame_range=frame_range,
    mode=mode,
    navg=navg,
    window_length=window_length,
    parallel=parallel,
    GPU=GPU
)

# Function to adjust contrast
def adjust_contrast(image, contrast_factor):
    """
    Adjust the contrast of an image.
    
    Parameters:
        image (np.ndarray): The input image to adjust.
        contrast_factor (float): Factor by which to adjust the contrast.
        
    Returns:
        np.ndarray: The adjusted image.
    """
    mean_intensity = np.mean(image)
    adjusted = contrast_factor * (image - mean_intensity) + mean_intensity
    adjusted = np.clip(adjusted, -0.0075, 0.0075)  # Keep within range
    return adjusted

# Interactive viewer for frames
def display_frame(frame_index, frames, contrast_factor=1.0, cmap='gray'):
    """
    Display a single frame with adjustable contrast and colormap.
    
    Parameters:
        frame_index (int): Index of the frame to display.
        frames (np.ndarray): Array of frames.
        contrast_factor (float): Contrast adjustment factor.
        cmap (str): Colormap to use for display.
    """
    plt.figure(figsize=(9.5, 9.5 * frames.shape[1] / frames.shape[2]))
    image = frames[frame_index]

    # Adjust contrast
    adjusted_image = adjust_contrast(image, contrast_factor)

    # Display the image
    plt.imshow(adjusted_image, cmap=cmap, vmin=-0.0075, vmax=0.0075)
    plt.title(f"Frame {frame_index}")
    plt.axis('off')
    plt.show()

def create_interactive_viewer(frames):
    """
    Create an interactive viewer for frames with contrast adjustment and colormap selection.
    
    Parameters:
        frames (np.ndarray): Array of frames.
    """
    interact(display_frame,
             frame_index=widgets.IntSlider(min=0, max=len(frames) - 1, step=1, value=0),
             frames=fixed(frames),
             contrast_factor=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Contrast'),
             cmap=widgets.Dropdown(options=['gray', 'viridis', 'plasma', 'magma', 'cividis'], value='gray', description='Colormap'))

# Create interactive viewers
print("Interactive Frame Slider:")
create_interactive_viewer(frames)

# Viewers for candidate detection and particle fitting
def display_candidates(frame_index, frames, contrast_factor=1.0, cmap='gray'):
    """
    Display a frame with candidates for particle detection.
    
    Parameters:
        frame_index (int): Index of the frame to display.
        frames (np.ndarray): Array of frames.
        contrast_factor (float): Contrast adjustment factor.
        cmap (str): Colormap to use for display.
    """
    plt.figure(figsize=(9.5, 9.5 * frames.shape[1] / frames.shape[2]))
    image = frames[frame_index]

    # Adjust contrast
    adjusted_image = adjust_contrast(image, contrast_factor)

    # Detect and display candidates
    # Use the detect module here if needed; this is a placeholder for illustration
    plt.imshow(adjusted_image, cmap=cmap, vmin=-0.0075, vmax=0.0075)
    plt.scatter(np.random.randint(0, frames.shape[2], 10),  # Example random candidates
                np.random.randint(0, frames.shape[1], 10),
                color='red', s=10)
    plt.title(f"Frame {frame_index} - Candidates")
    plt.axis('off')
    plt.show()

def create_candidate_viewer(frames):
    """
    Create an interactive viewer for particle detection candidates.
    
    Parameters:
        frames (np.ndarray): Array of frames.
    """
    interact(display_candidates,
             frame_index=widgets.IntSlider(min=0, max=len(frames) - 1, step=1, value=0),
             frames=fixed(frames),
             contrast_factor=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Contrast'),
             cmap=widgets.Dropdown(options=['gray', 'viridis', 'plasma', 'magma', 'cividis'], value='gray', description='Colormap'))

print("Interactive Candidate Viewer:")
create_candidate_viewer(frames)

def display_fits(frame_index, frames, contrast_factor=1.0, cmap='gray'):
    """
    Display a frame with particle fits.
    
    Parameters:
        frame_index (int): Index of the frame to display.
        frames (np.ndarray): Array of frames.
        contrast_factor (float): Contrast adjustment factor.
        cmap (str): Colormap to use for display.
    """
    plt.figure(figsize=(9.5, 9.5 * frames.shape[1] / frames.shape[2]))
    image = frames[frame_index]

    # Adjust contrast
    adjusted_image = adjust_contrast(image, contrast_factor)

    # Fit and display particles
    # Use the fit module here if needed; this is a placeholder for illustration
    plt.imshow(adjusted_image, cmap=cmap, vmin=-0.0075, vmax=0.0075)
    plt.scatter(np.random.randint(0, frames.shape[2], 10),  # Example random fits
                np.random.randint(0, frames.shape[1], 10),
                color='green', s=10)
    plt.title(f"Frame {frame_index} - Fits")
    plt.axis('off')
    plt.show()

def create_fit_viewer(frames):
    """
    Create an interactive viewer for particle fits.
    
    Parameters:
        frames (np.ndarray): Array of frames.
    """
    interact(display_fits,
             frame_index=widgets.IntSlider(min=0, max=len(frames) - 1, step=1, value=0),
             frames=fixed(frames),
             contrast_factor=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Contrast'),
             cmap=widgets.Dropdown(options=['gray', 'viridis', 'plasma', 'magma', 'cividis'], value='gray', description='Colormap'))

print("Interactive Fit Viewer:")
create_fit_viewer(frames)
